In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
# import shared_utils

# from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

In [ ]:
## parameters cell
itp_id = 300

In [ ]:
analysis_date = dt.date(2022, 10, 13)

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

# Big Blue Bus

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for Big Blue Bus
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

In [ ]:
rt_day.rt_trips.route_short_name.unique()

In [ ]:
lincoln_rts = ['3', 'R3']

## Afternoon Peak

* apparently no usable data earlier in the day, to investigate

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['R3'])

In [ ]:
# rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=['3'])

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m

In [ ]:
m = rt_day.segment_speed_map(how='average')

In [ ]:
m

In [ ]:
# rt_day.detailed_map_view >> filter(_.shape_id == '26032') >> arrange(_.stop_sequence)

In [ ]:
rt_day.rt_trips.direction = rt_day.rt_trips.direction.map(lambda x: 'Eastbound' if x =='Southbound' else x) # rename partial data trip to match

In [ ]:
(rt_day._filter(rt_day.rt_trips)
 >> group_by(_.route_short_name, _.shape_id, _.direction)
 >> summarize(mean_mph = _.mean_speed_mph.mean())
)

# Initial Thoughts

* Rapid 3 often slower than standard 3
* Rapid 3 not currently using Lincoln Blvd S of Manchester, routing is Manchester to Sepulveda
    * Previous iterations of route did -- unsure if change was due to congestion or desire to serve destinations along Manchester and the LAX City Bus Center
    * This new routing also seems to better serve the future LAX Airport Metro Connector
    * Let's assume the new routing for this analysis -- these segments have fairly high speeds anyhow
* No bus lanes installed on SHN portion yet (existing lane in City of Santa Monica only)

## Speedmap Interpretation (PM Peak)

* _29 minutes_ for the average R3 trip to travel 1.5 miles from Broadway Street, past Venice Bl, to SR90
    * at the 20th percentile for each segment, over 45 minutes to go that same distance!
    
## Corridor Delay Interpretation

* Define corridor as Ozone to Manchester, starting at beginning of SHN portion and ending where BBB 3/R3 routes diverge
* Weekday average schedule delay is _400 minutes_, weekday average speed-based delay is _1226 minutes_
* If bus lanes achieve 16mph average, a speed already exceeded on less congested parts of route, riders would save 20 hours per day _and_ Big Blue Bus would be able to add 20 additional trips with the same number of operating hours - further improving the rider experience and reducing travel times

In [ ]:
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=lincoln_rts, shape_ids=['26032'])

In [ ]:
rt_day.chart_variability()

In [ ]:
rt_day.set_filter(start_time='15:00', end_time='19:00', route_names=lincoln_rts, shape_ids=['26031'])

In [ ]:
rt_day.chart_variability()

# Corridor Delay

In [ ]:
# corridor = gpd.read_file('./corridors_100buses/lincoln_ozone_manchester.geojson')

In [ ]:
sub_corridor = gpd.read_file('./corridors_100buses/lincoln_ozone_lucille.geojson')

In [ ]:
rt_day.add_corridor(sub_corridor)

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
rt_day.set_filter(route_names=lincoln_rts)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day._filter(rt_day.endpoint_delay_view).delay_seconds.median()

In [ ]:
890/60

## Averaged Metrics

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
full_wk = np.arange('2022-10-08', '2022-10-15', dtype='datetime64[D]') # 7-day week

In [ ]:
weekdays = np.arange('2022-10-10', '2022-10-15', dtype='datetime64[D]') # weekdays

In [ ]:
def average_metrics(itp_id, date_range, pbar, filter_dict=None):

    schedule_metrics = []
    speed_metrics = []
    for date in date_range:
        date = date.astype(dt.date)
        try:
            rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
            rt_day.add_corridor(sub_corridor)
            if filter_dict:
                rt_day.set_filter(**filter_dict)
            _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
            metrics = rt_day.corridor_metrics()
            schedule_metrics += [metrics['schedule_metric_minutes']]
            speed_metrics += [metrics['speed_metric_minutes']]
            print(f'complete for date: {date}')
        except Exception as e:
            print(f'failed for date: {date}')
            print(e)
            continue
    schedule_metric = np.round(np.mean(schedule_metrics), 0)
    speed_metric = np.round(np.mean(speed_metrics), 0)
    return {'avg_schedule_metric_minutes': schedule_metric,
           'avg_speed_metric_minutes': speed_metric,
           'all_schedule': schedule_metrics,
           'all_speed': speed_metrics}

In [ ]:
pbar = tqdm()

In [ ]:
metrics_unfiltered = average_metrics(300, weekdays, pbar)

In [ ]:
metrics_unfiltered

## What about midday? 

In [ ]:
rt_day.analysis_date

In [ ]:
rt_day.set_filter(start_time='09:00', end_time='16:00', route_names=['3', 'R3'])

In [ ]:
m = rt_day.segment_speed_map(how='average')

In [ ]:
m

In [ ]:
filter_dict = {'start_time': '09:00', 'end_time': '16:00'}

In [ ]:
midday_delay = average_metrics(300, weekdays, pbar, filter_dict)

In [ ]:
midday_delay